In [1]:
import warnings, os
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from glob import glob


import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from aggmap import AggMap, loadmap
import seaborn as sns


import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score
from sklearn.metrics import auc as calculate_auc


## LGR, RF, KNN, ROF, XGBoost

In [2]:
from sklearn.ensemble import RandomForestClassifier
from rotation_forest import RotationForestClassifier

In [3]:
def score(dfr):
    y_true = dfr.y_true
    y_score = dfr.y_score
    y_pred = dfr.y_score.round()

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    acc = (tp + tn) / sum([tn, fp, fn, tp])
    auc = roc_auc_score(y_true, y_score)
    ber =  (fp / (fp + tn) + fn / (tp + fn)) * 0.5

    print('acc: %.3f, roc-auc: %.3f, BER: %.3f' % (acc,auc,ber))

    return acc, auc


def get_best_params(X, y):
    clf = RotationForestClassifier()
    parameters = {'max_depth': [4, 6, 9],
                  'n_estimators': [10, 50, 100]}

    grid = GridSearchCV(clf, parameters, scoring='roc_auc', n_jobs=9, iid=False, cv=5, verbose=2)
    gres = grid.fit(X, y)
    return grid.best_params_

In [4]:
lst = glob('/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-G/*.h5')

n_fold = 5

res = []
for p in lst:
    
    print('#'*50 + ' %s ' % p + '#'*50 )
    dfx = pd.read_hdf(p, key = 'expression')
    dfy = pd.read_hdf(p, key = 'labels')
    print(p, dfy.value_counts().to_dict())

    outer = KFold(n_splits = n_fold, shuffle = True, random_state = 123)
    outer_idx = list(outer.split(dfx, dfy))

    fold_all = []
    for i, idx in enumerate(outer_idx):

        fold_num = "fold_%s" % str(i).zfill(2) 

        train_idx, test_idx = idx

        # subset feature selection by the fold change
        dfxy_train = dfx.iloc[train_idx].join(dfy.iloc[train_idx])
        abds = dfxy_train.groupby(dfy.name).mean()
        fc = abs(abds.iloc[0] - abds.iloc[1])
        selected = fc[fc > 0.5].index

        trainX = dfx[selected].iloc[train_idx].values
        trainY = dfy.iloc[train_idx].values

        testX = dfx[selected].iloc[test_idx].values
        testY = dfy.iloc[test_idx].values

        ## training
        print("\nInput train and test X shape is %s, %s  \n" % (trainX.shape,  testX.shape))
        print("Getting the best parameters by gridsearch \n")
        #best_params = get_best_params(trainX, trainY)
        best_params = {'n_estimators':50, 'n_features_per_subset':10}
        
        clf = RotationForestClassifier(**best_params, n_jobs=20, verbose=1)
        clf.fit(trainX, trainY)

        ## evaluation
        y_true = testY
        y_score = clf.predict_proba(testX)[:,1]
        dfr = pd.DataFrame([y_true, y_score]).T
        dfr.columns = ['y_true', 'y_score']
        dfr.index = dfy.iloc[test_idx].index
        acc, auc = score(dfr)
        
        fold_all.append(auc)
        res.append([auc, i, p, best_params])
        
    print('The auc score for %s is %s.' % (p, np.mean(fold_all)))


################################################## /home/shenwanxiang/Research/AggMapNet_dataset/TCGA-G/O_z-score_train_LIHC_grade.h5 ##################################################
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-G/O_z-score_train_LIHC_grade.h5 {0: 250, 1: 124}

Input train and test X shape is (299, 2260), (75, 2260)  

Getting the best parameters by gridsearch 



[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:   18.2s
[Parallel(n_jobs=20)]: Done  50 out of  50 | elapsed:   43.7s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.7s
[Parallel(n_jobs=20)]: Done  50 out of  50 | elapsed:    2.9s finished


acc: 0.613, roc-auc: 0.583, BER: 0.424

Input train and test X shape is (299, 2854), (75, 2854)  

Getting the best parameters by gridsearch 



[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:   27.4s
[Parallel(n_jobs=20)]: Done  50 out of  50 | elapsed:   59.7s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.7s
[Parallel(n_jobs=20)]: Done  50 out of  50 | elapsed:    3.2s finished


acc: 0.533, roc-auc: 0.464, BER: 0.528

Input train and test X shape is (299, 1785), (75, 1785)  

Getting the best parameters by gridsearch 



...

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:   27.8s
[Parallel(n_jobs=20)]: Done  50 out of  50 | elapsed:  1.1min finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done  50 out of  50 | elapsed:    3.1s finished


acc: 0.632, roc-auc: 0.668, BER: 0.354

Input train and test X shape is (426, 3157), (106, 3157)  

Getting the best parameters by gridsearch 



[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:   32.5s
[Parallel(n_jobs=20)]: Done  50 out of  50 | elapsed:  1.2min finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.9s


acc: 0.557, roc-auc: 0.633, BER: 0.443
The auc score for /home/shenwanxiang/Research/AggMapNet_dataset/TCGA-G/O_z-score_train_LGG_grade.h5 is 0.6712292527971315.


[Parallel(n_jobs=20)]: Done  50 out of  50 | elapsed:    3.4s finished


In [7]:
dfres = pd.DataFrame(res, columns = ['score', 'fold', 'task','best_params'])
dfres.to_csv('./FS_RoationForest_5FCV_results_grade.csv')
dfres.groupby('task').mean().sort_values('score',ascending=False)

,score,fold
task,,
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-G/O_z-score_train_UCEC_grade.h5,0.770515,2
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-G/O_z-score_train_LGG_grade.h5,0.671229,2
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-G/O_z-score_train_STAD_grade.h5,0.636267,2
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-G/O_z-score_test_HNSC_grade.h5,0.587229,2
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-G/O_z-score_train_LIHC_grade.h5,0.550368,2
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-G/O_z-score_train_KIRC_grade.h5,0.513963,2
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-G/O_z-score_train_CESC_grade.h5,0.492634,2
/home/shenwanxiang/Research/AggMapNet_dataset/TCGA-G/O_z-score_train_PAAD_grade.h5,0.490246,2
